# 計算ターゲットの使用

Azure Machine Learning SDKを使用して多くの実験を実行しましたが、それらはすべてローカルコンピューティング（この場合はAzure Machine LearningノートブックVM）で実行されました。次に、クラウドコンピューティングを活用して、コンピューティングコンテキストのスケーラビリティを向上させる方法を確認します。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用のデータを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。以下のセルを実行して、このデータセットを作成します（以前のラボで作成した場合、コードは新しいバージョンを作成します）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset ready.')

## 計算ターゲットを作成する

多くの場合、ローカルのコンピューティングリソースでは、大量のデータを処理する必要がある複雑な実験や長時間実行する実験を処理するには不十分な場合があります。また、クラウドでコンピューティングリソースを動的に作成して使用する機能を活用することもできます。

Azure MLは、さまざまな計算ターゲットをサポートしています。これらのターゲットは、ワークスペースで定義し、実験の実行に使用できます。リソースの使用時にのみリソースの支払いを行います。最初の演習でワークスペースをセットアップしたときに、** aml-cluster **というトレーニングクラスターを作成したので、それを使用してトレーニング実験を実行できるように、存在することを確認します（そうでない場合は作成します）。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# Verify that cluster exists
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

## リモートコンピュートで実験を実行する

コンピューティングを作成したので、それを使用して実験を実行できます。次のコードは、実験ファイル用のフォルダーを作成します（以前のラボから既に存在している可能性がありますが、とにかく実行してください！）

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

次に、実験用のPythonスクリプトファイルを作成します。これにより、前の実習で使用したスクリプトが上書きされます。

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、作成したコンピューティングで実験を実行する準備が整いました。 

> **Note**: コンテナイメージをconda環境で構築し、クラスターノードを起動し、スクリプトを実行する前にイメージをデプロイする必要があるため、実験にはかなり長い時間がかかります。糖尿病のトレーニングスクリプトのような単純な実験では、これは非効率的に見えるかもしれません。しかし、数時間かかるより複雑な実験を実行する必要があると想像してください-よりスケーラブルな計算を動的に作成すると、全体の時間が大幅に短縮される可能性があります。

In [ ]:
from azureml.core import Environment, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
diabetes_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies
diabetes_packages = CondaDependencies.create(conda_packages=['pip==19.3.1','scikit-learn','ipykernel','matplotlib', 'pandas'],
                                             pip_packages=['azureml-sdk','pyarrow'])

# Add the dependencies to the environment
diabetes_env.python.conda_dependencies = diabetes_packages

# Register the environment (just in case previous lab wasn't completed)
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')],
              compute_target = cluster_name, # Use the compute target created previously
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()

実験の実行を待っている間に、上記のウィジェットまたは[Azure Machine Learning studio](https://ml.azure.com)で計算のステータスを確認できます。以下のコードを使用して、コンピューティングのステータスを確認することもできます。ステータスが*定常*から*サイズ変更*に変わるまでにはしばらく時間がかかることに注意してください（今はコーヒーブレイクを取るのに良い時間かもしれません！）

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

実験が終了すると、実験の実行によって生成されたメトリックとファイルを取得できます。今回は、ファイルにはイメージの構築とコンピューティングの管理のためのログが含まれます。

In [ ]:
run.wait_for_completion()

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

これで、実験によって訓練されたモデルを登録できます。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Azure ML compute'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

>**詳しくは**: Azure Machine Learningの計算ターゲットの詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)を参照してください。